In [11]:
import os
import polars as pl
import pandas as pd
import numpy as np
from tqdm import tqdm
from time import time

In [12]:
# Setup paths
PROJ_DIR = os.path.dirname(os.getcwd())
DATA_DIR = os.path.join(PROJ_DIR, "jane-street-real-time-market-data-forecasting")
OUTPUT_DIR = os.path.join(PROJ_DIR, "feature_enhanced_data")

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Feature column definitions
FEATURE_COLS = [f'feature_{i:02d}' for i in range(79)]

### Null flags + forward then backward fill data

In [13]:
def handle_nulls(df: pd.DataFrame, feature_cols: list) -> pd.DataFrame:
    """
    Strategy:
    1. Create null flags for ALL features (not just ones with nulls)
    2. Forward fill within each symbol_id group
    3. Backward fill remaining nulls (handles first rows)
    """
    print("\n🔍 Processing null values...")
    
    # Store original shape for validation
    original_shape = df.shape
    
    # Create null flags for ALL features
    null_features = []
    for col in tqdm(feature_cols, desc="Creating null flags"):
        # Create flag column regardless of null presence
        flag_col = f'{col}_is_null'
        df[flag_col] = df[col].isnull().astype(np.int8)
        
        # Still track statistics for logging
        null_count = df[col].isnull().sum()
        if null_count > 0:
            null_features.append(col)
            print(f"  {col}: {null_count:,} nulls ({(null_count/len(df))*100:.2f}%)")
    
    # First forward fill within each symbol_id group
    print("\n📈 Forward filling values within symbol groups...")
    ffill_start = time()
    df[feature_cols] = df.groupby('symbol_id')[feature_cols].ffill()
    ffill_time = time() - ffill_start
    print(f"Forward fill completed in {ffill_time:.2f} seconds")
    
    # Handle remaining nulls (first rows) with backward fill
    remaining_nulls = df[feature_cols].isnull().sum()
    if remaining_nulls.any():
        print("\n⚠️ Backward filling remaining nulls (first rows)...")
        bfill_start = time()
        df[feature_cols] = df.groupby('symbol_id')[feature_cols].bfill()
        bfill_time = time() - bfill_start
        print(f"Backward fill completed in {bfill_time:.2f} seconds")
        
        # Check if any nulls still remain (this would happen if entire column is null for a symbol)
        final_nulls = df[feature_cols].isnull().sum()
        if final_nulls.any():
            print("\n⚠️ Warning: Some columns still have nulls after forward and backward fill.")
            print("These are likely entire null columns for some symbols.")
            # Fill these with 0 or another appropriate value
            zero_fill_start = time()
            df[feature_cols] = df[feature_cols].fillna(0)
            zero_fill_time = time() - zero_fill_start
            print(f"Zero fill completed in {zero_fill_time:.2f} seconds")
    
    # Validate processing
    assert df[feature_cols].isnull().sum().sum() == 0, "Found remaining nulls after processing"
    assert df.shape[0] == original_shape[0], "Row count changed during processing"
    
    return df

### Create new data set

In [ ]:
print("🚀 Starting data enhancement process...")
start_time = time()

# Load training data
print("Reading training data...")
train = pl.scan_parquet(os.path.join(DATA_DIR, "train.parquet")).\
    select(
        pl.int_range(pl.len(), dtype=pl.UInt64).alias("id"),
        pl.all(),
    )

print("Converting to pandas DataFrame...")
with tqdm(total=1, desc="Loading data") as pbar:
    train_df = train.collect().to_pandas()
    pbar.update(1)

# Record initial shape and memory usage
initial_shape = train_df.shape
initial_mem = train_df.memory_usage().sum() / 1024 / 1024
print(f"\nInitial DataFrame shape: {initial_shape}")
print(f"Initial memory usage: {initial_mem:.2f} MB")

# Process the data
enhanced_train = handle_nulls(train_df, FEATURE_COLS)

# Convert back to Polars for efficient partitioned saving
print("\n💾 Converting to Polars DataFrame for partitioned saving...")
pl_enhanced = pl.from_pandas(enhanced_train)

# Calculate partition sizes
n_partitions = 20
rows_per_partition = len(pl_enhanced) // n_partitions
print(f"Saving {n_partitions} partitions with ~{rows_per_partition:,} rows each")

# Create output directory structure
output_dir = os.path.join(OUTPUT_DIR, "enhanced_train.parquet")
os.makedirs(output_dir, exist_ok=True)

# Save partitioned data
print("\n💾 Saving partitioned enhanced data...")
for i in tqdm(range(n_partitions), desc="Saving partitions"):
    start_idx = i * rows_per_partition
    end_idx = None if i == n_partitions - 1 else (i + 1) * rows_per_partition
    
    partition = pl_enhanced.slice(start_idx, rows_per_partition if end_idx else len(pl_enhanced) - start_idx)
    partition_path = os.path.join(output_dir, f"partition_id={i}")
    os.makedirs(partition_path, exist_ok=True)
    partition.write_parquet(os.path.join(partition_path, "part-0.parquet"))

end_time = time()
print(f"\n✨ Processing completed in {end_time - start_time:.2f} seconds")

# Print sample of changes
print("\nSample of enhanced data (first 5 rows of first feature):")
sample = pl_enhanced.select(['feature_00', 'feature_00_is_null']).head(5)
print(sample)

🚀 Starting data enhancement process...
Reading training data...
Converting to pandas DataFrame...


Loading data: 100%|██████████| 1/1 [00:56<00:00, 56.15s/it]



Initial DataFrame shape: (47127338, 94)
Initial memory usage: 16584.38 MB

🔍 Processing null values...


Creating null flags:   1%|▏         | 1/79 [00:00<00:46,  1.66it/s]

  feature_00: 3,182,052 nulls (6.75%)


...

Creating null flags:  99%|█████████▊| 78/79 [01:48<00:01,  1.08s/it]

  feature_77: 20,043 nulls (0.04%)


Creating null flags: 100%|██████████| 79/79 [01:48<00:00,  1.38s/it]

  feature_78: 20,043 nulls (0.04%)

📈 Forward filling values within symbol groups...


Forward fill completed in 288.62 seconds

⚠️ Backward filling remaining nulls (first rows)...
Backward fill completed in 216.95 seconds

💾 Saving enhanced data to /monfs01/projects/ys68/JaneStreet-Kaggle/feature_enhanced_data/enhanced_train.parquet

✨ Processing completed in 922.12 seconds

Sample of enhanced data (first 5 rows of first feature):
   feature_00  feature_00_is_null
0    1.161135                   1
1    1.157178                   1
2    2.085530                   1
3    1.237014                   1
4    1.358605                   1
